In [2]:
!pip install uv
!uv pip install tensorflow
!uv pip install fredapi
!uv pip install yfinance
!uv pip install python-dotenv

Using Python 3.11.13 environment at: /usr
Audited 1 package in 139ms
Using Python 3.11.13 environment at: /usr
Audited 1 package in 85ms
Using Python 3.11.13 environment at: /usr
Audited 1 package in 87ms
Using Python 3.11.13 environment at: /usr
Audited 1 package in 88ms


In [3]:
import os
import datetime
import numpy as np
import pandas as pd
import yfinance as yf
from fredapi import Fred
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticker

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization, MultiHeadAttention
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [4]:
# --- 設定とAPIキー ---
from dotenv import load_dotenv

# .env ファイルから環境変数をロード
load_dotenv()

# --- 設定とAPIキー ---
# FRED APIキーを環境変数から取得
fred_api_key = os.getenv('FRED_API_KEY')
if fred_api_key is None:
    raise ValueError("FRED_API_KEY が .env ファイルに設定されていません。")

fred = Fred(api_key=fred_api_key) # APIキーを初期化


# データの開始日と終了日
START_DATE = '2000-01-01'
END_DATE = datetime.date.today().strftime('%Y-%m-%d') # 現在の日付まで

In [5]:
# --- Google Driveを安全にマウント ---
# COLAB_ENVIRONMENTフラグを利用して、Colab環境でのみGoogle Drive関連の処理を行う
try:
    from google.colab import drive
    COLAB_ENVIRONMENT = True
    MOUNT_POINT = '/content/drive'
except ImportError:
    COLAB_ENVIRONMENT = False
    MOUNT_POINT = None # Colab環境でなければマウントポイントは不要

def safe_drive_mount(mount_point):
    """Google Driveをマウントし、既にマウントされている場合はその旨を伝える。"""
    try:
        drive.mount(mount_point)
        print('[INFO] Google Driveをマウントしました。')
        return True # マウント成功
    except Exception as e:
        msg = str(e)
        if 'already mounted' in msg or 'must not already contain files' in msg:
            print('[INFO] Google Driveはすでにマウントされています。')
            return True # すでにマウント済みと判断
        else:
            print(f'[ERROR] Google Driveのマウント中にエラーが発生しました: {e}')
            return False # マウント失敗

In [7]:
"""

# --- 基本ディレクトリ設定 ---
if COLAB_ENVIRONMENT:
    # Google Colab環境の場合
    if safe_drive_mount(MOUNT_POINT):
        BASE_DIR = os.path.join(MOUNT_POINT, 'MyDrive','S&P500_prediction')
    else:
        # マウントに失敗した場合のフォールバック
        print("[WARNING] Google Driveのマウントに失敗したため、ローカルディレクトリを使用します。")
        BASE_DIR = './'
else:
    # Google Colab環境ではない場合（ローカルPCなど）
    print("[INFO] Google Colab環境ではないため、ローカルディレクトリを使用します。")
    BASE_DIR = './' # カレントディレクトリに作成

os.makedirs(BASE_DIR, exist_ok=True) # ディレクトリが存在しない場合は作成
print(f"[INFO] データ保存先ディレクトリ: {BASE_DIR}")
"""

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[INFO] Google Driveをマウントしました。
[INFO] データ保存先ディレクトリ: /content/drive/MyDrive/S&P500_prediction


# 補完済みの情報の取得

In [14]:
# --- CSVファイルからのデータロード ---
# ファイル名の設定
file_name_csv = 'market_data.csv'

# 保存パスの結合
file_path_csv = os.path.join(BASE_DIR, file_name_csv)

print(f"\n--- CSVファイルからデータをロード中: {file_path_csv} ---")
loaded_data_csv = pd.DataFrame()
try:
    # CSVファイルをロード
    # index_col=0 は、CSVの最初の列（日付）をDataFrameのインデックスとして読み込むことを指定します。
    # parse_dates=True は、インデックスの列を日付型に変換することを指定します。
    loaded_data_csv = pd.read_csv(file_path_csv, index_col=0, parse_dates=True)
    print("[INFO] CSVファイルからデータが正常にロードされました。")

    # ロードしたデータの最初の数行を表示して確認
    print("\nロードされたデータの最初の5行:")
    display(loaded_data_csv)

    # ロードしたデータの情報（データ型など）を確認
    print("\nロードされたデータの概要:")
    print(loaded_data_csv.info())

except FileNotFoundError:
    print(f"[ERROR] ファイルが見つかりません: {file_path_csv}")
    print("指定されたパスにCSVファイルが存在するか確認してください。")
except Exception as e:
    print(f"[ERROR] CSVファイルのロード中にエラーが発生しました: {e}")


--- CSVファイルからデータをロード中: /content/drive/MyDrive/S&P500_prediction/market_data.csv ---
[INFO] CSVファイルからデータが正常にロードされました。

ロードされたデータの最初の5行:


,Industrial Production,Unemployment Rate,CPI (All Urban),Federal Funds Rate,10-Year Treasury Yield,2-Year Treasury Yield,VIX Index,Oil Price WTI,Consumer Sentiment,M2 Money Stock,...,Utilities Select Sector SPDR Fund,Real Estate Select Sector SPDR Fund,Communication Services Select Sector SPDR Fund,iShares MSCI EAFE ETF,iShares MSCI Emerging Markets ETF,FXI (China Large-Cap ETF),EWJ (Japan ETF),CBOE Interest Rate 10-Year T-Note,CBOE Interest Rate 2-Year T-Note,US Dollar Index (DXY)
2000-01-01,91.4092,4.0,169.30,5.45,6.58,6.38,24.21,25.56,112.0,4667.6,...,11.261680,21.580572,46.785187,22.604351,7.340264,11.356076,43.690880,6.548,5.270,100.22
2000-01-02,91.4092,4.0,169.30,5.45,6.58,6.38,24.21,25.56,112.0,4667.6,...,11.261680,21.580572,46.785187,22.604351,7.340264,11.356076,43.690880,6.548,5.270,100.22
2000-01-03,91.4092,4.0,169.30,5.45,6.58,6.38,24.21,25.56,112.0,4667.6,...,11.261680,21.580572,46.785187,22.604351,7.340264,11.356076,43.690880,6.548,5.270,100.22
2000-01-04,91.4092,4.0,169.30,5.45,6.49,6.30,27.01,25.56,112.0,4667.6,...,10.921969,21.580572,46.785187,22.604351,7.340264,11.356076,42.510036,6.485,5.270,100.41
2000-01-05,91.4092,4.0,169.30,5.45,6.62,6.38,26.41,24.65,112.0,4667.6,...,11.197584,21.580572,46.785187,22.604351,7.340264,11.356076,41.329216,6.599,5.270,100.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,103.5948,4.1,320.58,4.33,4.24,3.72,16.73,66.30,52.2,21942.0,...,81.660000,41.420000,108.530000,89.390000,48.240000,36.760000,74.970000,4.230,4.190,96.88
2025-07-01,103.5948,4.1,320.58,4.33,4.26,3.78,16.83,66.30,52.2,21942.0,...,81.940000,41.700000,107.760000,89.240000,48.330000,36.830000,74.420000,4.251,4.225,96.82
2025-07-02,103.5948,4.1,320.58,4.33,4.30,3.78,16.64,66.30,52.2,21942.0,...,81.230000,41.780000,107.450000,89.500000,48.540000,36.690000,74.460000,4.293,4.223,96.78
2025-07-03,103.5948,4.1,320.58,4.33,4.30,3.78,16.64,66.30,52.2,21942.0,...,81.840000,41.800000,108.040000,89.520000,48.760000,36.270000,74.600000,4.348,4.240,97.18



ロードされたデータの概要:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 9317 entries, 2000-01-01 to 2025-07-04
Columns: 186 entries, Industrial Production to US Dollar Index (DXY)
dtypes: float64(186)
memory usage: 13.3 MB
None


# グローバル変数

In [11]:
# --- グローバル変数設定 ---
WINDOW_SIZE = 252 #（約1年間の営業日 # 過去n日間のデータを使用
# PRED_HORIZON = 1 # 1日後の予測 (元の設定)
MAX_PRED_HORIZON = 30 # 1日から30日後までを予測

TEST_SIZE_RATIO = 0.1 # テストデータの割合
VALID_SIZE_RATIO = 0.1 # 検証データの割合
BATCH_SIZE = 8
EPOCHS = 1 # エポック数を増加させる可能性あり
LEARNING_RATE = 1e-3

current_target_column = 'SP500_Return'
PCA_N_COMPONENTS = 0.95 # PCAで保持する分散の割合

print(f"ターゲットカラム: {current_target_column}")
print(f"PCA情報保存率の閾値: {PCA_N_COMPONENTS}")

ターゲットカラム: SP500_Return
PCA情報保存率の閾値: 0.95


In [7]:
!uv pip install plotly

Using Python 3.11.13 environment at: /usr
Audited 1 package in 163ms


In [8]:
import plotly.express as px
import gc

In [9]:
# --- ディレクトリ設定 ---
if COLAB_ENVIRONMENT:
    # Google Colab環境の場合
    if safe_drive_mount(MOUNT_POINT):
        BASE_DIR = os.path.join(MOUNT_POINT, 'MyDrive','S&P500_prediction')
    else:
        # マウントに失敗した場合のフォールバック
        print("[WARNING] Google Driveのマウントに失敗したため、ローカルディレクトリを使用します。")
        BASE_DIR = './'
else:
    # Google Colab環境ではない場合（ローカルPCなど）
    print("[INFO] Google Colab環境ではないため、ローカルディレクトリを使用します。")
    BASE_DIR = './' # カレントディレクトリに作成

os.makedirs(BASE_DIR, exist_ok=True) # ディレクトリが存在しない場合は作成
print(f"[INFO] データ保存先ディレクトリ: {BASE_DIR}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[INFO] Google Driveをマウントしました。
[INFO] データ保存先ディレクトリ: /content/drive/MyDrive/S&P500_prediction


In [2]:
# --- 前処理済みデータファイルのパス定義 ---
PREPROCESSED_DATA_DIR = os.path.join(BASE_DIR, 'Model_training_in_progress','preprocessed_data')
os.makedirs(PREPROCESSED_DATA_DIR, exist_ok=True)

X_TRAIN_PATH = os.path.join(PREPROCESSED_DATA_DIR, 'X_train_pca.npy')
Y_TRAIN_PATH = os.path.join(PREPROCESSED_DATA_DIR, 'y_train_scaled.npy')
DATES_TRAIN_PATH = os.path.join(PREPROCESSED_DATA_DIR, 'dates_train.csv')

X_VALID_PATH = os.path.join(PREPROCESSED_DATA_DIR, 'X_valid_pca.npy')
Y_VALID_PATH = os.path.join(PREPROCESSED_DATA_DIR, 'y_valid_scaled.npy')
DATES_VALID_PATH = os.path.join(PREPROCESSED_DATA_DIR, 'dates_valid.csv')

X_TEST_PATH = os.path.join(PREPROCESSED_DATA_DIR, 'X_test_pca.npy')
Y_TEST_PATH = os.path.join(PREPROCESSED_DATA_DIR, 'y_test_scaled.npy')
DATES_TEST_PATH = os.path.join(PREPROCESSED_DATA_DIR, 'dates_test.csv')

# PCAオブジェクトとスケーラーオブジェクトも保存・ロードできるようにパスを定義
PCA_MODEL_PATH = os.path.join(PREPROCESSED_DATA_DIR, 'pca_model.pkl')
FEATURE_SCALER_PATH = os.path.join(PREPROCESSED_DATA_DIR, 'feature_scaler.pkl')
TARGET_SCALER_PATH = os.path.join(PREPROCESSED_DATA_DIR, 'target_scaler.pkl')

# scikit-learnオブジェクトの保存・ロードにはjoblibが必要
import joblib

# --- 前処理済みデータのロードまたは新規作成 ---
print("\n--- 前処理済みデータのロードまたは新規作成 ---")

# 全てのファイルが存在するかチェック
all_preprocessed_files_exist = (
    os.path.exists(X_TRAIN_PATH) and os.path.exists(Y_TRAIN_PATH) and os.path.exists(DATES_TRAIN_PATH) and
    os.path.exists(X_VALID_PATH) and os.path.exists(Y_VALID_PATH) and os.path.exists(DATES_VALID_PATH) and
    os.path.exists(X_TEST_PATH) and os.path.exists(Y_TEST_PATH) and os.path.exists(DATES_TEST_PATH) and
    os.path.exists(PCA_MODEL_PATH) and os.path.exists(FEATURE_SCALER_PATH) and os.path.exists(TARGET_SCALER_PATH)
)

if all_preprocessed_files_exist:
    print("[INFO] 既存の前処理済みデータが見つかりました。ロードします。")
    X_train_pca = np.load(X_TRAIN_PATH)
    y_train_scaled = np.load(Y_TRAIN_PATH)
    dates_train_full = pd.read_csv(DATES_TRAIN_PATH, index_col=0, parse_dates=True).index

    X_valid_pca = np.load(X_VALID_PATH)
    y_valid_scaled = np.load(Y_VALID_PATH)
    dates_valid_full = pd.read_csv(DATES_VALID_PATH, index_col=0, parse_dates=True).index

    X_test_pca = np.load(X_TEST_PATH)
    y_test_scaled = np.load(Y_TEST_PATH)
    dates_test_full = pd.read_csv(DATES_TEST_PATH, index_col=0, parse_dates=True).index

    # スケーラーとPCAモデルもロード
    pca = joblib.load(PCA_MODEL_PATH)
    feature_scaler = joblib.load(FEATURE_SCALER_PATH)
    target_scaler = joblib.load(TARGET_SCALER_PATH)

    feature_cols = feature_scaler.feature_names_in_ # feature_scalerから元の特徴量名を取得
    original_feature_dim = len(feature_cols)

    print(f"ロードされた訓練データ形状: {X_train_pca.shape}, ターゲット形状: {y_train_scaled.shape}")
    print(f"情報保存率:{PCA_N_COMPONENTS}_PCA後の特徴量次元数: {pca.n_components_} (元の次元: {original_feature_dim})")

else:
    print("[INFO] 前処理済みデータが見つかりません。新規に作成します。")
    # --- CSVファイルからのデータロード (前のコードからコピー) ---
    file_name_csv = 'market_data.csv'
    # BASE_DIRの親ディレクトリからロードするようにパスを調整
    file_path_csv = os.path.join(BASE_DIR, file_name_csv)

    print(f"\n--- CSVファイルからデータをロード中: {file_path_csv} ---")

    loaded_data_csv = None
    try:
        loaded_data_csv = pd.read_csv(file_path_csv, index_col=0, parse_dates=True)
        print("[INFO] CSVファイルからデータが正常にロードされました。")
        print("\nロードされたデータの最初の5行:")
        display(loaded_data_csv.head())
        print("\nロードされたデータの概要:")
        print(loaded_data_csv.info())
    except FileNotFoundError:
        print(f"[ERROR] ファイルが見つかりません: {file_path_csv}")
        print("指定されたパスにCSVファイルが存在するか確認してください。")
    except Exception as e:
        print(f"[ERROR] CSVファイルのロード中にエラーが発生しました: {e}")

    if loaded_data_csv is None:
        print("[ERROR] データロードに失敗したため、処理を終了します。")
        exit()

    print("\n--- 1. ロードしたデータに対して最終的な前処理を行います ---")

    if 'S&P500' not in loaded_data_csv.columns:
        raise ValueError("S&P500 カラムがロードされたデータフレームに見つかりません。")
    loaded_data_csv['S&P500'] = loaded_data_csv['S&P500'].astype(np.float32)
    loaded_data_csv[current_target_column] = loaded_data_csv['S&P500'].pct_change(1).shift(-1)

    initial_rows_before_target_dropna = loaded_data_csv.shape[0]
    loaded_data_csv.dropna(subset=[current_target_column], inplace=True)
    rows_deleted_for_target = initial_rows_before_target_dropna - loaded_data_csv.shape[0]
    if rows_deleted_for_target > 0:
        print(f"ターゲット ({current_target_column}) のNaNを削除した結果、{rows_deleted_for_target} 行が削除されました。")

    initial_rows_after_target_dropna = loaded_data_csv.shape[0]
    loaded_data_csv.dropna(how='all', inplace=True)
    rows_deleted_all_nan_after_target = initial_rows_after_target_dropna - loaded_data_csv.shape[0]
    if rows_deleted_all_nan_after_target > 0:
        print(f"ターゲットNaN削除後に全てNaNの行を削除した結果、{rows_deleted_all_nan_after_target} 行が削除されました。")

    feature_cols_all = [col for col in loaded_data_csv.columns if col != current_target_column]
    full_data_df = loaded_data_csv[feature_cols_all + [current_target_column]].copy()

    initial_rows_final = full_data_df.shape[0]
    full_data_df.dropna(inplace=True)
    rows_deleted_final_features = initial_rows_final - full_data_df.shape[0]
    if rows_deleted_final_features > 0:
        print(f"特徴量内の欠損値 ({rows_deleted_final_features} 行) を削除しました。")

    full_data_df = full_data_df.astype(np.float32)

    print(f"最終的なデータ形状 (特徴量とターゲット): {full_data_df.shape}")
    print(f"最終的なデータセットの欠損値数 (最終確認):\n{full_data_df.isnull().sum().sum()}")
    if full_data_df.isnull().sum().sum() > 0:
        raise ValueError("データ前処理後に欠損値が残っています。データ処理を確認してください。")
    print("データ前処理完了。")

    del loaded_data_csv
    gc.collect()

    # --- 3. データを訓練・検証・テストに分割 (時系列順) ---
    print("\n--- 3. データ分割 (時系列順) ---")

    feature_cols = [col for col in full_data_df.columns if col != current_target_column]
    original_feature_dim = len(feature_cols)
    print(f"PCA適用前の特徴量次元数: {original_feature_dim}")

    train_size = int(len(full_data_df) * (1 - TEST_SIZE_RATIO - VALID_SIZE_RATIO))
    valid_size = int(len(full_data_df) * VALID_SIZE_RATIO)
    test_size = len(full_data_df) - train_size - valid_size

    train_df = full_data_df.iloc[:train_size].copy()
    valid_df = full_data_df.iloc[train_size:train_size + valid_size].copy()
    test_df = full_data_df.iloc[train_size + valid_size:].copy()

    del full_data_df
    gc.collect()

    print(f"データ分割: 訓練={len(train_df)} 検証={len(valid_df)} テスト={len(test_df)}")

    dates_train_full = train_df.index.copy()
    dates_valid_full = valid_df.index.copy()
    dates_test_full = test_df.index.copy()

    # --- 4. 特徴量スケーリングとPCA適用 ---
    print("\n--- 4. 特徴量スケーリングとPCA適用 ---")

    feature_scaler = StandardScaler()
    X_train_scaled = feature_scaler.fit_transform(train_df[feature_cols])
    X_valid_scaled = feature_scaler.transform(valid_df[feature_cols])
    X_test_scaled = feature_scaler.transform(test_df[feature_cols])

    y_train_raw = train_df[[current_target_column]].values
    y_valid_raw = valid_df[[current_target_column]].values
    y_test_raw = test_df[[current_target_column]].values

    del train_df
    del valid_df
    del test_df
    gc.collect()

    pca = PCA(n_components=PCA_N_COMPONENTS)
    X_train_pca = pca.fit_transform(X_train_scaled).astype(np.float32)
    X_valid_pca = pca.transform(X_valid_scaled).astype(np.float32)
    X_test_pca = pca.transform(X_test_scaled).astype(np.float32)

    del X_train_scaled
    del X_valid_scaled
    del X_test_scaled
    gc.collect()

    print(f"情報保存率:{PCA_N_COMPONENTS}_PCA後の特徴量次元数: {pca.n_components_} (元の次元: {original_feature_dim})")

    target_scaler = StandardScaler()
    y_train_scaled = target_scaler.fit_transform(y_train_raw).astype(np.float32)
    y_valid_scaled = target_scaler.transform(y_valid_raw).astype(np.float32)
    y_test_scaled = target_scaler.transform(y_test_raw).astype(np.float32)

    del y_train_raw, y_valid_raw, y_test_raw
    gc.collect()

    # --- 前処理済みデータの保存 ---
    print("\n[INFO] 前処理済みデータをGoogle Driveに保存中...")
    np.save(X_TRAIN_PATH, X_train_pca)
    np.save(Y_TRAIN_PATH, y_train_scaled)
    pd.DataFrame(index=dates_train_full).to_csv(DATES_TRAIN_PATH)

    np.save(X_VALID_PATH, X_valid_pca)
    np.save(Y_VALID_PATH, y_valid_scaled)
    pd.DataFrame(index=dates_valid_full).to_csv(DATES_VALID_PATH)

    np.save(X_TEST_PATH, X_test_pca)
    np.save(Y_TEST_PATH, y_test_scaled)
    pd.DataFrame(index=dates_test_full).to_csv(DATES_TEST_PATH)

    # スケーラーとPCAモデルも保存
    joblib.dump(pca, PCA_MODEL_PATH)
    joblib.dump(feature_scaler, FEATURE_SCALER_PATH)
    joblib.dump(target_scaler, TARGET_SCALER_PATH)
    print("[INFO] 前処理済みデータの保存が完了しました。")


# --- 4.1. PCAローディングの詳細な出力 ---
print("\n--- 各主成分に最も寄与する上位10特徴量 ---")

if pca.components_ is not None and len(feature_cols) == pca.components_.shape[1]:
    for i, pc_loadings in enumerate(pca.components_):
        pc_index = i + 1
        loadings_series = pd.Series(pc_loadings, index=feature_cols)
        top_10_features = loadings_series.abs().nlargest(10).index
        top_10_loadings_values = loadings_series[top_10_features]

        if i < len(pca.explained_variance_ratio_):
            contribution_ratio = pca.explained_variance_ratio_[i] * 100
        else:
            contribution_ratio = 0.0

        print(f"\n--- 主成分 PC{pc_index} (寄与率: {contribution_ratio:.2f}%) ---")
        for feature_name, loading_value in top_10_loadings_values.items():
            print(f"   - {feature_name}: {loading_value:.4f}")
else:
    print("PCA components_のデータが利用できないか、特徴量名の長さが一致しません。")
    print("PCA components_ shape:", pca.components_.shape if pca.components_ is not None else "None")
    print("feature_cols length:", len(feature_cols))

# --- 5. シーケンスデータの準備 (`prepare_sequences`) をジェネレータ化 ---
print("\n--- 5. シーケンスデータを準備中 (ジェネレータ使用) ---")

def prepare_sequences_generator(features_array, targets_array, dates_series, window_size, max_pred_horizon):
    for i in range(len(features_array) - window_size - max_pred_horizon + 1):
        target_returns_slice = targets_array[i + window_size : i + window_size + max_pred_horizon]
        if len(target_returns_slice) == max_pred_horizon:
            yield (features_array[i:(i + window_size)].astype(np.float32),
                   target_returns_slice.astype(np.float32))

output_signature = (
    tf.TensorSpec(shape=(WINDOW_SIZE, X_train_pca.shape[1]), dtype=tf.float32),
    tf.TensorSpec(shape=(MAX_PRED_HORIZON,), dtype=tf.float32)
)

# TensorFlow Datasetの作成は、データがロードまたは作成された後に実行
train_dataset = tf.data.Dataset.from_generator(
    lambda: prepare_sequences_generator(X_train_pca, y_train_scaled.flatten(), dates_train_full, WINDOW_SIZE, MAX_PRED_HORIZON),
    output_signature=output_signature
)
train_dataset = train_dataset.shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

valid_dataset = tf.data.Dataset.from_generator(
    lambda: prepare_sequences_generator(X_valid_pca, y_valid_scaled.flatten(), dates_valid_full, WINDOW_SIZE, MAX_PRED_HORIZON),
    output_signature=output_signature
)
valid_dataset = valid_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_generator(
    lambda: prepare_sequences_generator(X_test_pca, y_test_scaled.flatten(), dates_test_full, WINDOW_SIZE, MAX_PRED_HORIZON),
    output_signature=output_signature
)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# ここで元のNumPy配列はTensorFlow Datasetに変換されたので、メモリ解放
#del X_train_pca, y_train_scaled, dates_train_full
#del X_valid_pca, y_valid_scaled, dates_valid_full
#del X_test_pca, y_test_scaled, dates_test_full
gc.collect()

print("TensorFlow Datasetの作成完了。")

# --- 7. PCA Loadingの可視化 (Plotly HTML出力) ---
print("\n--- 7. PCA Components Loadingsの可視化 (Plotly HTML出力) ---")

if pca.components_ is not None and len(feature_cols) == pca.components_.shape[1]:
    pca_loadings_df = pd.DataFrame(
        pca.components_,
        columns=feature_cols,
        index=[f'PC{i+1}' for i in range(pca.n_components_)]
    ).astype(np.float32)

    fig = px.imshow(pca_loadings_df,
                    labels=dict(x="Original Features", y="Principal Components", color="Loading Weight"),
                    x=feature_cols,
                    y=[f'PC{i+1}' for i in range(pca.n_components_)],
                    color_continuous_scale='RdBu',
                    color_continuous_midpoint=0,
                    title='PCA Components Loadings: Contribution of Original Features to Principal Components',
                    width=1800,
                    height=1200
                    )

    fig.update_traces(hovertemplate='Original Feature: %{x}<br>Principal Component: %{y}<br>Loading: %{z:.2f}<extra></extra>')
    fig.update_xaxes(tickangle=90, tickfont=dict(size=8), automargin=True)
    fig.update_yaxes(tickfont=dict(size=10), automargin=True)

    html_filename = os.path.join(BASE_DIR, 'pca_loadings_heatmap.html')
    fig.write_html(html_filename)

    print(f"PCA Loadingsヒートマップが '{html_filename}' として保存されました。ブラウザで開いて確認してください。")
    print("ブラウザでは、ズーム、パン、ホバーによる詳細情報の確認が可能です。")

    del pca_loadings_df
    del fig
    gc.collect()
else:
    print("PCA components_のデータが利用できないか、特徴量名の長さが一致しません。")
    print("PCA components_ shape:", pca.components_.shape if pca.components_ is not None else "None")
    print("feature_cols length:", len(feature_cols))

NameError: name 'os' is not defined

In [1]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, Callback # Callbackをインポート

# Mixed Precisionを有効にするポリシーを設定
tf.keras.mixed_precision.set_global_policy('mixed_float16')
print("Mixed Precision (mixed_float16) を有効にしました。")

# iTransformerBlock クラスの定義
# Transformerモデルに、self-Attention機構を入れることで、iTransfomerに
class iTransformerBlock(tf.keras.layers.Layer):
    """
    iTransformerモデルの基本ブロックを定義します。
    標準的なTransformerブロックのSelf-Attention機構を、
    「時間軸を特徴量、特徴量次元をトークン」として転置して適用します。
    """
    def __init__(self, d_model, num_heads, dropout_rate, time_steps, **kwargs):
        """
        iTransformerBlockの初期化
        Args:
            d_model (int): Attention層の内部次元。特徴量（変数）がこの次元に埋め込まれる。
            num_heads (int): Multi-Head Attentionのヘッド数。
            dropout_rate (float): ドロップアウト率。
            time_steps (int): 入力シーケンスの時系列長 (T)。FFNの出力次元として使用。
        """
        super().__init__(**kwargs)
        self.d_model = d_model
        self.num_heads = num_heads
        self.dropout_rate = dropout_rate
        self.time_steps = time_steps # T (WINDOW_SIZE)

        # Multi-Head Self-Attention層の定義
        # key_dim=d_model: 各ヘッドのキー/クエリ/バリューの次元（特徴量トークンの埋め込み次元）
        self.attention = MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.attn_dropout = Dropout(dropout_rate) # Attention後のドロップアウト
        self.attn_norm = LayerNormalization(epsilon=1e-6) # Attention後のLayerNormalization

        # Feed-Forward Network (FFN) の深層化
        # iTransformer論文の解釈に基づき、FFNは時系列特徴量（T次元）を変換する。
        # 深層化のために中間層を追加。
        self.ffn = tf.keras.Sequential([
            Dense(d_model * 4, activation='relu'), # 最初のDense層 (活性化関数ReLU)
            Dense(d_model * 2, activation='relu'), # <<== 追加：FFNの中間層（深層化のため）
            Dense(time_steps) # 最終的なDense層（出力次元は時系列長Tのまま）
        ])
        self.ffn_dropout = Dropout(dropout_rate) # FFN後のドロップアウト
        self.ffn_norm = LayerNormalization(epsilon=1e-6) # FFN後のLayerNormalization

    def call(self, inputs, training=False):
        """
        iTransformerBlockのフォワードパス
        Args:
            inputs (tf.Tensor): 入力テンソル。形状は (batch_size, T, D_emb)。
                                D_embは特徴量埋め込み後の次元（通常d_model）。
            training (bool): 訓練モードかどうかを示すフラグ。ドロップアウトの挙動に影響。
        Returns:
            tf.Tensor: 処理後の出力テンソル。形状は入力と同じ (batch_size, T, D_emb)。
        """
        # iTransformerの核心: 変数軸と時間軸を転置
        # 入力: (batch_size, 時系列長T, 埋め込み特徴量D_emb)
        # 転置後: (batch_size, 埋め込み特徴量D_emb, 時系列長T)
        # これにより、Attentionが「D_emb個のトークン（変数）、各トークンはT次元の特徴量を持つ」として機能。
        inputs_transposed = tf.transpose(inputs, perm=[0, 2, 1])

        # Multi-Head Self-Attentionの適用
        # クエリ(Q), キー(K), バリュー(V)は全て転置された入力。
        # Attentionは埋め込み特徴量D_emb軸（トークン軸）に対して行われる。
        attn_output = self.attention(inputs_transposed, inputs_transposed)
        attn_output = self.attn_dropout(attn_output, training=training)
        # 残差接続 (Residual connection) と層正規化 (LayerNormalization)
        attn_output = self.attn_norm(inputs_transposed + attn_output) # (batch_size, D_emb, T)

        # Feed-Forward Network (FFN) の適用
        # FFNは、Attention後の各トークン（各変数）のT次元の特徴ベクトルを変換する。
        ffn_output = self.ffn(attn_output) # (batch_size, D_emb, T)
        ffn_output = self.ffn_dropout(ffn_output, training=training)
        # 残差接続と層正規化
        output = self.ffn_norm(attn_output + ffn_output) # (batch_size, D_emb, T)

        # 最終出力を元の形状 (batch_size, T, D_emb) に転置し直す
        return tf.transpose(output, perm=[0, 2, 1])


def build_itransformer_model(time_steps, feature_dim, d_model, num_heads, num_blocks, dropout_rate, output_horizon):
    """
    iTransformerモデルを構築する。
    Args:
        time_steps (int): シーケンス長 (T)
        feature_dim (int): 特徴量の次元数 (D)
        d_model (int): Attention層の内部次元
        num_heads (int): Multi-Head Attentionのヘッド数
        num_blocks (int): iTransformerブロックの数
        dropout_rate (float): ドロップアウト率
        output_horizon (int): モデルの出力次元数 (例: 1日後から30日後までの予測なので 30)
    Returns:
        tf.keras.Model: 構築されたiTransformerモデル
    """
    inputs = Input(shape=(time_steps, feature_dim), dtype=tf.float32)
    x = inputs

    # Feature embedding (各変数を d_model に線形変換) の深層化
    # 各時間ステップの各特徴量に対して適用されるようにTimeDistributedを使用
    x = tf.keras.layers.TimeDistributed(Dense(d_model, activation='relu'))(x) # <<== 活性化関数を追加 + 1層目
    x = tf.keras.layers.TimeDistributed(Dense(d_model))(x) # <<== 追加：2層目の埋め込み層

    # num_blocks が外から渡されるため、これで制御
    for _ in range(num_blocks): # <<== NUM_BLOCKS の値で深さが変わる
        x = iTransformerBlock(d_model=d_model, num_heads=num_heads,
                              dropout_rate=dropout_rate, time_steps=time_steps)(x)

    # Global Average Pooling (時間軸T方向に平均を取る)
    # (batch_size, T, d_model) -> (batch_size, d_model)
    x = tf.keras.layers.GlobalAveragePooling1D()(x)

    # 最終的な予測層: 出力次元を output_horizon に変更
    # ここも深層化するためにDense層を追加
    x = Dense(d_model // 2, activation='relu')(x) # <<== 追加：予測ヘッドの中間層
    outputs = Dense(output_horizon)(x) # 1日後から MAX_PRED_HORIZON 日後までのリターンを予測

    model = Model(inputs=inputs, outputs=outputs)
    return model

# --- カスタムコールバックの定義 ---
class CheckpointAndResumeCallback(Callback):
    def __init__(self, base_dir, model_name='itransformer_model', initial_epoch=0):
        super().__init__()
        self.base_dir = base_dir
        self.model_path = os.path.join(base_dir, f'{model_name}.keras') # Keras 3.x形式
        self.epoch_file = os.path.join(base_dir, 'last_epoch.txt')
        self.initial_epoch = initial_epoch

    def on_epoch_end(self, epoch, logs=None):
        # モデルの保存
        self.model.save(self.model_path)
        print(f"\n[INFO] モデルをエポック {epoch+1} で保存しました: {self.model_path}")

        # 現在のエポック数を保存
        with open(self.epoch_file, 'w') as f:
            f.write(str(epoch + 1)) # 次に開始するエポック数を記録
        print(f"[INFO] 現在のエポック数 {epoch+1} を保存しました: {self.epoch_file}")

    def on_train_begin(self, logs=None):
        # 訓練開始時に初期エポックをログに出力
        if self.initial_epoch > 0:
            print(f"[INFO] 訓練をエポック {self.initial_epoch} から再開します。")


# --- 7. モデルの構築と訓練 ---
print("--- モデル構築と訓練 ---")
feature_dim_after_pca = pca.n_components_ # PCA後の特徴量次元

# iTransformerモデルのハイパーパラメータ

# D_MODEL: モデルの埋め込み次元（embedding dimension）または特徴量次元。
#          Transformerの各層における入力と出力の次元数を示す。
#          この値が大きいほど、モデルはより多くの情報を表現できるが、計算コストとメモリ使用量が増加。
#          通常、Attentionメカニズムの入力・出力の次元にも影響します。
#          【D_MODELを増やすことの影響】
#          モデルが各ステップで処理できる情報の「幅」が広がり、よりリッチな表現を学習できるようになります。
#          これにより、複雑な特徴量を捉える能力が向上する可能性があります。
D_MODEL = 512

# NUM_HEADS: Multi-Head AttentionにおけるAttentionヘッドの数。
#            Multi-Head Attentionは、異なる表現サブスペースからの情報を並行して学習するために、
#            Attentionメカニズムを複数回（NUM_HEADSの数だけ）並列に実行します。
#            これにより、モデルは異なる種類の関係性やパターンを同時に捉えることができます。
#            D_MODELは通常、NUM_HEADSで割り切れる必要があります (D_MODEL % NUM_HEADS == 0)。
#            【NUM_HEADSを増やすことの影響】
#            モデルが同時に異なる側面に注意を向けられるようになり、データの多様な関係性をより深く探求できます。
#            これにより、情報の処理能力と表現学習の「深さ」が向上する可能性があります。
NUM_HEADS = 16

# NUM_BLOCKS: EncoderまたはDecoderブロック（iTransformerの場合は主にEncoderブロック）の数。
#             Transformerモデルは通常、複数のIdenticalなブロックを積み重ねて構成されます。
#             この値が大きいほど、モデルはより深いネットワークになり、複雑なパターンを学習する能力が高まりますが、
#             勾配消失/爆発の問題や学習時間の増加、過学習のリスクも高まります。
#             【NUM_BLOCKSを増やすことの影響】
#             モデルの「深さ」が直接的に増します。より多くの層を重ねることで、
#             入力データからより抽象的で高レベルな特徴を段階的に抽出し、複雑な階層的関係性を学習する能力が高まります。
#             これが、ご指摘の「学習の深さが変わる」という点に最も直接的に関連します。
NUM_BLOCKS = 4

# DROPOUT_RATE: ドロップアウト層のドロップアウト率。
#               過学習を防ぐための正則化手法の一つです。
#               訓練中に、各層のニューロンの一部をランダムに「ドロップアウト」（無効化）します。
#               この値は、ドロップアウトされるニューロンの割合を示します（例: 0.1は10%のニューロンを無効化）。
#               ドロップアウトは訓練時にのみ適用され、推論時には適用されません。
#               【DROPOUT_RATEの役割】
#               「過学習を防ぐ」ために使用されます。
#               モデルが特定の訓練データに過度に依存するのを防ぎ、汎化能力を高める効果があります。
#               特にD_MODEL, NUM_HEADS, NUM_BLOCKSを増やすことでモデルの表現力が高まり、
#               過学習のリスクも増大するため、ドロップアウトのような正則化手法の重要性が増します。
DROPOUT_RATE = 0.2

# --- 学習再開ロジック ---
model = None
initial_epoch = 0
checkpoint_model_path = os.path.join(BASE_DIR, 'itransformer_model.keras') # Keras 3.x形式のパス
last_epoch_file = os.path.join(BASE_DIR, 'last_epoch.txt')

print("\n--- 学習再開チェック ---")
if os.path.exists(checkpoint_model_path) and os.path.exists(last_epoch_file):
    try:
        # モデルのロード
        model = tf.keras.models.load_model(checkpoint_model_path,
                                           custom_objects={'iTransformerBlock': iTransformerBlock})
        print(f"[INFO] 既存のモデルをロードしました: {checkpoint_model_path}")

        # 最終エポック数のロード
        with open(last_epoch_file, 'r') as f:
            initial_epoch = int(f.read())
        print(f"[INFO] 学習をエポック {initial_epoch} から再開します。")

        # 最適化関数と学習率を再設定 (ロードしたモデルはコンパイル済みの状態が復元されるが、学習率を確実に適用するため)
        model.compile(optimizer=Adam(learning_rate=LEARNING_RATE, clipnorm=1.0), loss='mse')

    except Exception as e:
        print(f"[ERROR] モデルまたはエポック数のロード中にエラーが発生しました: {e}")
        print("[INFO] 新しいモデルとして学習を開始します。")
        model = None # エラー時は新しくモデルを構築
        initial_epoch = 0
else:
    print("[INFO] 保存されたモデルが見つかりませんでした。新しいモデルとして学習を開始します。")

# モデルがロードされなかった場合、新しくモデルを構築
if model is None:
    model = build_itransformer_model(
        time_steps=WINDOW_SIZE,
        feature_dim=feature_dim_after_pca,
        d_model=D_MODEL,
        num_heads=NUM_HEADS,
        num_blocks=NUM_BLOCKS,
        dropout_rate=DROPOUT_RATE,
        output_horizon=MAX_PRED_HORIZON
    )
    # 新しいモデルの場合はここでコンパイル
    model.compile(optimizer=Adam(learning_rate=LEARNING_RATE, clipnorm=1.0), loss='mse')

model.summary()

# コールバックの設定
early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=12, min_lr=5e-8, verbose=1)

# カスタムコールバックのインスタンス化
checkpoint_callback = CheckpointAndResumeCallback(
    base_dir=BASE_DIR,
    initial_epoch=initial_epoch # initial_epochを渡すことでログに出力される
)


# 訓練データセットの総サンプル数（シーケンス数）を計算
# len(features_array) - window_size - max_pred_horizon + 1
num_train_sequences = len(X_train_pca) - WINDOW_SIZE - MAX_PRED_HORIZON + 1
num_valid_sequences = len(X_valid_pca) - WINDOW_SIZE - MAX_PRED_HORIZON + 1


# steps_per_epoch を計算（訓練データ）
train_steps_per_epoch = int(np.ceil(num_train_sequences / BATCH_SIZE))

# validation_steps も計算（検証データ）
# 検証データは通常エポックごとに全て処理されるため、train_steps_per_epoch と同様に計算します。
validation_steps_per_epoch = int(np.ceil(num_valid_sequences / BATCH_SIZE))


print("モデル訓練を開始します...")
history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=valid_dataset,
    callbacks=[early_stopping, reduce_lr, checkpoint_callback], # カスタムコールバックを追加
    verbose=1,
    initial_epoch=initial_epoch # ここで学習再開エポックを指定
    # ここに steps_per_epoch と validation_steps を追加
    steps_per_epoch=train_steps_per_epoch,
    validation_steps=validation_steps_per_epoch
)
print("モデル訓練完了。")

# --- ここから新しい評価と可視化のコードを追加 ---

# 評価に必要なライブラリをインポート
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error

print("\n--- モデルの評価と予測 ---")

y_pred_scaled = model.predict(test_dataset)

y_true_scaled_list = []
for _, labels in test_dataset.unbatch().as_numpy_iterator():
    y_true_scaled_list.append(labels)
y_true_scaled = np.array(y_true_scaled_list)

# y_pred_scaled の形状を調整 (MAX_PRED_HORIZONが1の場合、(N, 1)から(N,)にflattenする)
# predict()の出力は(N, MAX_PRED_HORIZON)ですが、inverse_transformは(N, 1)のような2D配列を期待します。
# y_true_scaled も(N, MAX_PRED_HORIZON)の形状なので、直接inverse_transformに渡します。
y_pred = target_scaler.inverse_transform(y_pred_scaled)
y_true = target_scaler.inverse_transform(y_true_scaled)

print(f"元のスケールに戻した予測値の最初の5つ: {y_pred[:5].flatten()}")
print(f"元のスケールに戻した実際の値の最初の5つ: {y_true[:5].flatten()}")

rmse = np.sqrt(mean_squared_error(y_true, y_pred))
mae = mean_absolute_error(y_true, y_pred)

print(f"\n評価指標:")
print(f"RMSE (Root Mean Squared Error): {rmse:.4f}")
print(f"MAE (Mean Absolute Error): {mae:.4f}")

print("\n--- 訓練履歴の可視化 ---")
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.grid(True)
plt.show()


print("\n--- 予測結果の可視化 ---")

# plot_dates の調整: y_true の長さと一致するようにスライスする
# test_dataset が prepare_sequences_generator から生成されているため、
# データの長さは len(original_data) - WINDOW_SIZE - MAX_PRED_HORIZON + 1 となります。
# dates_test_full からこの開始点以降の日付を取得します。
plot_dates_start_idx = WINDOW_SIZE # シーケンスの始まり
# plot_dates は y_true の各要素に対応する日付であるべきです。
# y_true はテストデータ全体のシーケンスの数になります。
plot_dates = dates_test_full[plot_dates_start_idx : plot_dates_start_idx + len(y_true)]

if len(plot_dates) != len(y_true):
    print(f"[WARNING] プロットする日付の数 ({len(plot_dates)}) と予測/実際の値の数 ({len(y_true)}) が一致しません。")
    print("日付の計算ロジックを確認してください。y_trueの長さに合わせて日付を調整します。")
    # ここでの再調整は、すでに上記の計算で試みられていますが、念のため。
    # 最も確実なのは、y_true_scaled_listを構築する際に、対応する日付も一緒に収集することです。
    # しかし、ここではシンプルにlen(y_true)に合わせてスライスします。
    plot_dates = dates_test_full[WINDOW_SIZE : WINDOW_SIZE + len(y_true)]


plt.figure(figsize=(16, 8))
plt.plot(plot_dates, y_true.flatten(), label='Actual Values', color='blue', alpha=0.7)
plt.plot(plot_dates, y_pred.flatten(), label='Predicted Values', color='red', alpha=0.7, linestyle='--')
plt.title('Actual vs. Predicted S&P500 Returns (Test Set)')
plt.xlabel('Date')
plt.ylabel('S&P500 Returns')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

print("\n--- 予測誤差の分布 ---")
errors = (y_true - y_pred).flatten()
plt.figure(figsize=(10, 6))
sns.histplot(errors, kde=True, bins=50)
plt.title('Distribution of Prediction Errors')
plt.xlabel('Prediction Error')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

print("\n--- 評価と可視化の完了 ---")

# 学習終了後、メモリ解放
# PCA, feature_scaler, target_scaler は既にjoblibで保存されているので、delしても問題ない
del X_train_pca, y_train_scaled, dates_train_full
del X_valid_pca, y_valid_scaled, dates_valid_full
del X_test_pca, y_test_scaled, dates_test_full
del pca
del feature_scaler
del target_scaler
gc.collect()

SyntaxError: invalid syntax. Perhaps you forgot a comma? (ipython-input-1-3744406409.py, line 272)

In [ ]:
# モデルの評価
print("--- モデル評価 ---")
train_loss = model.evaluate(train_dataset, verbose=0)
valid_loss = model.evaluate(valid_dataset, verbose=0)
test_loss = model.evaluate(test_dataset, verbose=0)

print(f"訓練セットのMSE: {train_loss:.4f}")
print(f"検証セットのMSE: {valid_loss:.4f}")
print(f"テストセットのMSE: {test_loss:.4f}")

# 予測の実行 (predsの形状が (N, MAX_PRED_HORIZON) になるため、.flatten() は削除)
train_preds = model.predict(train_dataset)
valid_preds = model.predict(valid_dataset)
test_preds = model.predict(test_dataset)

In [ ]:
train_steps_per_epoch = math.ceil(len(X_train_seq) / BATCH_SIZE) # X_train_seq が prepare_sequences で得られた配列と仮定
valid_steps_per_epoch = math.ceil(len(X_valid_seq) / BATCH_SIZE)
test_steps_per_epoch = math.ceil(len(X_test_seq) / BATCH_SIZE)


# 予測値を元のスケールに戻すヘルパー関数
def inverse_transform_predictions(scaled_preds, scaler):
    original_shape = scaled_preds.shape
    # (N, M) -> (N*M, 1) にreshape (StandardScalerが1次元入力を期待するため)
    flat_scaled_preds = scaled_preds.reshape(-1, 1)
    # 逆変換
    flat_original_preds = scaler.inverse_transform(flat_scaled_preds)
    # (N*M, 1) -> (N, M) にreshapeし直す
    original_preds = flat_original_preds.reshape(original_shape)
    return original_preds

# モデルの評価 (損失はスケーリングされたままのMSE)
print("--- モデル評価 (スケーリングされた損失) ---")
train_loss = model.evaluate(train_dataset, steps=train_steps_per_epoch, verbose=0)
valid_loss = model.evaluate(valid_dataset, steps=valid_steps_per_epoch, verbose=0)
test_loss = model.evaluate(test_dataset, steps=test_steps_per_epoch, verbose=0)

print(f"訓練セットのMSE (スケーリング済み): {train_loss:.4f}")
print(f"検証セットのMSE (スケーリング済み): {valid_loss:.4f}")
print(f"テストセットのMSE (スケーリング済み): {test_loss:.4f}")


# 予測の実行
print("--- 予測実行 ---")
train_preds_scaled = model.predict(train_dataset, steps=train_steps_per_epoch)
valid_preds_scaled = model.predict(valid_dataset, steps=valid_steps_per_epoch)
test_preds_scaled = model.predict(test_dataset, steps=test_steps_per_epoch)

print(f"訓練予測形状 (スケーリング済み): {train_preds_scaled.shape}")
print(f"検証予測形状 (スケーリング済み): {valid_preds_scaled.shape}")
print(f"テスト予測形状 (スケーリング済み): {test_preds_scaled.shape}")


# 予測値を元のスケールに戻す
print("--- 予測値の標準化解除 ---")
train_preds_original_scale = inverse_transform_predictions(train_preds_scaled, target_scaler)
valid_preds_original_scale = inverse_transform_predictions(valid_preds_scaled, target_scaler)
test_preds_original_scale = inverse_transform_predictions(test_preds_scaled, target_scaler)

print(f"訓練予測形状 (元のスケール): {train_preds_original_scale.shape}")
print(f"検証予測形状 (元のスケール): {valid_preds_original_scale.shape}")
print(f"テスト予測形状 (元のスケール): {test_preds_original_scale.shape}")

# 実際のターゲット値も元のスケールに戻す（評価やプロットのため）
print("--- 実際のターゲット値の標準化解除 ---")
# y_train_seq, y_valid_seq, y_test_seq は prepare_sequences で得られたスケーリング済みのターゲット配列
actual_train_returns_original_scale = inverse_transform_predictions(y_train_seq, target_scaler)
actual_valid_returns_original_scale = inverse_transform_predictions(y_valid_seq, target_scaler)
actual_test_returns_original_scale = inverse_transform_predictions(y_test_seq, target_scaler)

print(f"訓練実際のターゲット形状 (元のスケール): {actual_train_returns_original_scale.shape}")
print(f"検証実際のターゲット形状 (元のスケール): {actual_valid_returns_original_scale.shape}")
print(f"テスト実際のターゲット形状 (元のスケール): {actual_test_returns_original_scale.shape}")


# 元のスケールでの評価指標の計算 (例: RMSE, MAE)
print("--- 元のスケールでの評価指標 ---")

# 通常、評価は一番最初の予測ホライズン (H+1) で行われることが多いです
# もし複数のホライズンすべてで評価する場合は、ループで回す必要があります
horizon_to_evaluate = 0 # 最初の予測ホライズン (H+1)

if MAX_PRED_HORIZON > 0:
    # 訓練セット
    train_rmse = np.sqrt(mean_squared_error(actual_train_returns_original_scale[:, horizon_to_evaluate],
                                            train_preds_original_scale[:, horizon_to_evaluate]))
    train_mae = mean_absolute_error(actual_train_returns_original_scale[:, horizon_to_evaluate],
                                    train_preds_original_scale[:, horizon_to_evaluate])

    # 検証セット
    valid_rmse = np.sqrt(mean_squared_error(actual_valid_returns_original_scale[:, horizon_to_evaluate],
                                            valid_preds_original_scale[:, horizon_to_evaluate]))
    valid_mae = mean_absolute_error(actual_valid_returns_original_scale[:, horizon_to_evaluate],
                                    valid_preds_original_scale[:, horizon_to_evaluate])

    # テストセット
    test_rmse = np.sqrt(mean_squared_error(actual_test_returns_original_scale[:, horizon_to_evaluate],
                                           test_preds_original_scale[:, horizon_to_evaluate]))
    test_mae = mean_absolute_error(actual_test_returns_original_scale[:, horizon_to_evaluate],
                                  test_preds_original_scale[:, horizon_to_evaluate])

    print(f"訓練セットのRMSE (H+{horizon_to_evaluate+1}): {train_rmse:.4f}, MAE: {train_mae:.4f}")
    print(f"検証セットのRMSE (H+{horizon_to_evaluate+1}): {valid_rmse:.4f}, MAE: {valid_mae:.4f}")
    print(f"テストセットのRMSE (H+{horizon_to_evaluate+1}): {test_rmse:.4f}, MAE: {test_mae:.4f}")
else:
    print("MAX_PRED_HORIZON が0のため、RMSE/MAEは計算されませんでした。")


# これで、train_preds_original_scale, valid_preds_original_scale, test_preds_original_scale
# および actual_train_returns_original_scale, actual_valid_returns_original_scale, actual_test_returns_original_scale
# を使って、プロットや詳細な分析を行うことができます。

In [ ]:
# 結果の可視化
def plot_results_multi_horizon(history, y_true_seq, y_pred_seq, dates, set_name, pred_horizon_to_plot=None):
    """
    訓練履歴と予測結果をプロットする関数（多出力対応版）
    Args:
        pred_horizon_to_plot (int or list of int, optional): プロットしたい予測ホライズン。
                                                             Noneの場合、1日後の予測のみプロット。
    """
    if pred_horizon_to_plot is None:
        plot_horizons = [0] # 0-indexed で 1日後の予測
    elif isinstance(pred_horizon_to_plot, int):
        plot_horizons = [pred_horizon_to_plot - 1] # ユーザー入力は1-indexed
    else:
        plot_horizons = [h - 1 for h in pred_horizon_to_plot] # リストの場合も1-indexedを0-indexedに変換

    num_plots_per_horizon = 2 # 予測vs実測、累積リターン
    total_subplots = len(plot_horizons) * num_plots_per_horizon + 1 # 損失 + 各ホライズンの2つのプロット

    fig = plt.figure(figsize=(18, 6 * len(plot_horizons) + 6)) # 全体の図のサイズを調整

    # 1. 訓練履歴 (損失)
    plt.subplot(total_subplots, 1, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Mean Squared Error')
    plt.legend()
    plt.grid(True)

    current_subplot_idx = 2
    for p_idx in plot_horizons:
        # y_true_seq, y_pred_seq は (N, MAX_PRED_HORIZON) 形状
        # p_idx は 0-indexed
        y_true_single_horizon = y_true_seq[:, p_idx]
        y_pred_single_horizon = y_pred_seq[:, p_idx]

        horizon_label = f"{p_idx + 1}-Day Ahead" # プロット用ラベル (1-indexed)

        # 2. 予測 vs 実測値 (特定期間)
        plt.subplot(total_subplots, 1, current_subplot_idx)
        display_start_idx = max(0, len(y_true_single_horizon) - 500)
        display_dates = dates[display_start_idx:]
        display_y_true = y_true_single_horizon[display_start_idx:]
        display_y_pred = y_pred_single_horizon[display_start_idx:]

        plt.plot(display_dates, display_y_true, label=f'{set_name} Actual Returns', color='blue', alpha=0.7)
        plt.plot(display_dates, display_y_pred, label=f'{set_name} Predicted Returns', color='red', linestyle='--', alpha=0.7)
        plt.title(f'S&P 500 Daily Returns Prediction ({set_name} Set - {horizon_label} - Last 500 Days)')
        plt.xlabel('Date')
        plt.ylabel('Daily Return (%)')
        plt.legend()
        plt.grid(True)
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
        plt.gca().xaxis.set_major_locator(mticker.AutoLocator())
        plt.gcf().autofmt_xdate()
        current_subplot_idx += 1

        # 3. 累積リターン (特定期間)
        plt.subplot(total_subplots, 1, current_subplot_idx)
        cumulative_true = np.cumsum(display_y_true)
        cumulative_pred = np.cumsum(display_y_pred)

        # 開始点を0に正規化
        cumulative_true_normalized = cumulative_true - cumulative_true[0]
        cumulative_pred_normalized = cumulative_pred - cumulative_pred[0]

        plt.plot(display_dates, cumulative_true_normalized, label=f'{set_name} Actual Cumulative Returns', color='green', linewidth=2)
        plt.plot(display_dates, cumulative_pred_normalized, label=f'{set_name} Predicted Cumulative Returns', color='purple', linestyle='--', linewidth=1.5)
        plt.title(f'S&P 500 Cumulative Returns ({set_name} Set - {horizon_label} - Last 500 Days)')
        plt.xlabel('Date')
        plt.ylabel('Cumulative Return (%)')
        plt.legend()
        plt.grid(True)
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
        plt.gca().xaxis.set_major_locator(mticker.AutoLocator())
        plt.gcf().autofmt_xdate()
        current_subplot_idx += 1

    plt.tight_layout()
    plt.show()

def plot_direction_accuracy_multi_horizon(y_true_seq, y_pred_seq, set_name):
    """
    方向精度を計算し、各予測ホライズンごとにプロットする関数（多出力対応版）
    """
    num_horizons = y_true_seq.shape[1] # 予測ホライズンの数
    accuracies = []

    for i in range(num_horizons):
        y_true_single = y_true_seq[:, i]
        y_pred_single = y_pred_seq[:, i]

        true_direction = np.sign(y_true_single)
        pred_direction = np.sign(y_pred_single)
        correct_directions = np.sum(true_direction == pred_direction)
        total_predictions = len(y_true_single)
        accuracy = correct_directions / total_predictions * 100
        accuracies.append(accuracy)

    horizon_labels = [f'{i+1}D' for i in range(num_horizons)]

    plt.figure(figsize=(15, 6))
    bars = plt.bar(horizon_labels, accuracies, color='skyblue')
    plt.ylabel('Directional Accuracy (%)')
    plt.xlabel('Prediction Horizon')
    plt.title(f'Directional Accuracy Across Prediction Horizons ({set_name} Set)')
    plt.ylim(0, 100) # 0%から100%の範囲
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, yval + 1, round(yval, 2), ha='center', va='bottom') # accuracy値を表示
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()


# 訓練・検証・テストセットの結果をプロット
# 各ホライズン（1日後、5日後、30日後）の予測を個別にプロット
plot_results_multi_horizon(history, y_train_seq, train_preds, dates_train, 'Train', pred_horizon_to_plot=[1, 5, 30])
plot_results_multi_horizon(history, y_valid_seq, valid_preds, dates_valid, 'Validation', pred_horizon_to_plot=[1, 5, 30])
plot_results_multi_horizon(history, y_test_seq, test_preds, dates_test, 'Test', pred_horizon_to_plot=[1, 5, 30])

# 方向精度を全てのホライズンでプロット
plot_direction_accuracy_multi_horizon(y_train_seq, train_preds, 'Train')
plot_direction_accuracy_multi_horizon(y_valid_seq, valid_preds, 'Validation')
plot_direction_accuracy_multi_horizon(y_test_seq, test_preds, 'Test')

In [ ]:
TARGET_SCALING_FACTOR_GLOBAL = 100#10000


# 結果の可視化
def plot_results_multi_horizon(history, y_true_seq, y_pred_seq, dates, set_name, pred_horizon_to_plot=None, target_scaling_factor=100):
    """
    訓練履歴と予測結果をプロットする関数（多出力対応版）
    Args:
        history: モデルの訓練履歴オブジェクト (keras.callbacks.History)。
        y_true_seq (np.array): 実測値のシーケンスデータ (N, num_horizons)。
        y_pred_seq (np.array): 予測値のシーケンスデータ (N, num_horizons)。
        dates (pd.DatetimeIndex or list of datetime): 対応する日付データ。
        set_name (str): データセットの名前 ('Train', 'Validation', 'Test' など)。
        pred_horizon_to_plot (int or list of int, optional): プロットしたい予測ホライズン (1-indexed)。
                                                          Noneの場合、0-indexedで1日後の予測 (最初のホライズン) のみプロット。
        target_scaling_factor (int/float): preprocess_dataでターゲットをスケーリングした係数。
                                           これを元のスケールに戻すために使用。デフォルトは100 (元々%)。
    """
    if pred_horizon_to_plot is None:
        plot_horizons = [0] # 0-indexed で 1日後の予測（最初のホライズン）
    elif isinstance(pred_horizon_to_plot, int):
        plot_horizons = [pred_horizon_to_plot - 1] # ユーザー入力は1-indexedを0-indexedに変換
    else:
        # リストの場合も1-indexedを0-indexedに変換
        plot_horizons = [h - 1 for h in pred_horizon_to_plot if h - 1 < y_true_seq.shape[1]]
        if not plot_horizons:
            print(f"警告: 指定された pred_horizon_to_plot {pred_horizon_to_plot} は有効なホライズンではありません。")
            print(f"y_true_seq のホライズン数は {y_true_seq.shape[1]} です。")
            return

    num_plots_per_horizon = 2 # 予測vs実測、累積リターン
    total_subplots = len(plot_horizons) * num_plots_per_horizon + 1 # 損失 + 各ホライズンの2つのプロット

    fig = plt.figure(figsize=(18, 6 * len(plot_horizons) + 6)) # 全体の図のサイズを調整

    # 1. 訓練履歴 (損失)
    plt.subplot(total_subplots, 1, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    if 'val_loss' in history.history: # 検証損失がない場合もあるためチェック
        plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Mean Squared Error')
    plt.legend()
    plt.grid(True)

    current_subplot_idx = 2
    for p_idx in plot_horizons:
        # y_true_seq, y_pred_seq は (N, num_horizons) 形状
        # p_idx は 0-indexed

        # ★ ここでスケーリングを元に戻す ★
        y_true_single_horizon = y_true_seq[:, p_idx] / target_scaling_factor * 100 # 元の%表示に戻す
        y_pred_single_horizon = y_pred_seq[:, p_idx] / target_scaling_factor * 100 # 元の%表示に戻す

        # ホライズンのラベルは元の日数で表示 (例: 1-Day, 5-Day, 20-Day)
        # y_true_seqの2次元目のインデックスp_idxは0から始まるため、p_idx+1が実際の日数に対応
        horizon_label_days = p_idx + 1
        horizon_title = f"{horizon_label_days}-Day Ahead" # プロット用ラベル (1-indexed)

        # 2. 予測 vs 実測値 (特定期間)
        plt.subplot(total_subplots, 1, current_subplot_idx)
        # 最近の500日間のデータを表示
        display_start_idx = max(0, len(y_true_single_horizon) - 500)
        display_dates = dates[display_start_idx:]
        display_y_true = y_true_single_horizon[display_start_idx:]
        display_y_pred = y_pred_single_horizon[display_start_idx:]

        plt.plot(display_dates, display_y_true, label=f'{set_name} Actual Returns', color='blue', alpha=0.7)
        plt.plot(display_dates, display_y_pred, label=f'{set_name} Predicted Returns', color='red', linestyle='--', alpha=0.7)
        plt.title(f'S&P 500 Returns Prediction ({set_name} Set - {horizon_title} - Last 500 Days)')
        plt.xlabel('Date')
        plt.ylabel('Return (%)') # Y軸ラベルもReturn (%) に変更
        plt.legend()
        plt.grid(True)
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
        plt.gca().xaxis.set_major_locator(mticker.AutoLocator())
        plt.gcf().autofmt_xdate()
        current_subplot_idx += 1

        # 3. 累積リターン (特定期間)
        plt.subplot(total_subplots, 1, current_subplot_idx)
        # 累積リターンも元のパーセンテージスケールで計算
        cumulative_true = np.cumsum(display_y_true)
        cumulative_pred = np.cumsum(display_y_pred)

        # 開始点を0に正規化
        cumulative_true_normalized = cumulative_true - cumulative_true[0]
        cumulative_pred_normalized = cumulative_pred - cumulative_pred[0]

        plt.plot(display_dates, cumulative_true_normalized, label=f'{set_name} Actual Cumulative Returns', color='green', linewidth=2)
        plt.plot(display_dates, cumulative_pred_normalized, label=f'{set_name} Predicted Cumulative Returns', color='purple', linestyle='--', linewidth=1.5)
        plt.title(f'S&P 500 Cumulative Returns ({set_name} Set - {horizon_title} - Last 500 Days)')
        plt.xlabel('Date')
        plt.ylabel('Cumulative Return (%)')
        plt.legend()
        plt.grid(True)
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
        plt.gca().xaxis.set_major_locator(mticker.AutoLocator())
        plt.gcf().autofmt_xdate()
        current_subplot_idx += 1

    plt.tight_layout()
    plt.show()

def plot_direction_accuracy_multi_horizon(y_true_seq, y_pred_seq, set_name, target_scaling_factor=100):
    """
    方向精度を計算し、各予測ホライズンごとにプロットする関数（多出力対応版）
    Args:
        y_true_seq (np.array): 実測値のシーケンスデータ (N, num_horizons)。
        y_pred_seq (np.array): 予測値のシーケンスデータ (N, num_horizons)。
        set_name (str): データセットの名前 ('Train', 'Validation', 'Test' など)。
        target_scaling_factor (int/float): preprocess_dataでターゲットをスケーリングした係数。
                                           これを元のスケールに戻すために使用。デフォルトは100 (元々%)。
    """
    num_horizons = y_true_seq.shape[1] # 予測ホライズンの数
    accuracies = []

    for i in range(num_horizons):
        # ★ ここでスケーリングを元に戻す ★
        # 方向精度を計算する際には、絶対値のスケールは関係ないため、
        # スケーリングを元に戻す必要は厳密にはありません。
        # np.sign() は正、負、ゼロを判断するため、スケーリングの影響を受けません。
        # しかし、一貫性のためにここでも元のパーセンテージリターンに換算するコードは残しておきます。
        # 必要に応じて、y_true_seq[:, i] / target_scaling_factor * 100 を y_true_seq[:, i] に戻しても問題ありません。
        y_true_single = y_true_seq[:, i] # / target_scaling_factor * 100 # Directional accuracy doesn't need scaling
        y_pred_single = y_pred_seq[:, i] # / target_scaling_factor * 100 # Directional accuracy doesn't need scaling

        # ゼロ近傍の予測は方向を判断しにくいため、閾値を設けることも検討できますが、
        # np.sign()は0を0として扱うため、これは通常問題になりません。
        true_direction = np.sign(y_true_single)
        pred_direction = np.sign(y_pred_single)

        # 両方が0の場合も一致とみなされることに注意。
        # 例えば、真値が0.001で予測が-0.001の場合、np.sign()は1と-1になり不一致。
        # 真値が0で予測が0の場合、np.sign()は0と0になり一致。
        correct_directions = np.sum(true_direction == pred_direction)
        total_predictions = len(y_true_single)
        accuracy = correct_directions / total_predictions * 100
        accuracies.append(accuracy)

    # ホライズンラベルは1-indexedで表示
    horizon_labels = [f'{i+1}D' for i in range(num_horizons)]

    plt.figure(figsize=(15, 6))
    bars = plt.bar(horizon_labels, accuracies, color='skyblue')
    plt.ylabel('Directional Accuracy (%)')
    plt.xlabel('Prediction Horizon')
    plt.title(f'Directional Accuracy Across Prediction Horizons ({set_name} Set)')
    plt.ylim(0, 100) # 0%から100%の範囲
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, yval + 1, round(yval, 2), ha='center', va='bottom') # accuracy値を表示
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()


# 訓練・検証・テストセットの結果をプロット
# global TARGET_SCALING_FACTOR_GLOBAL を使用するか、preprocess_dataから返された値を使用
# ここでは例として TARGET_SCALING_FACTOR_GLOBAL を直接渡しています。

plot_results_multi_horizon(history, y_train_seq, train_preds, dates_train, 'Train',
                           pred_horizon_to_plot=[1, 5, 20], # 例として20日後までプロット
                           target_scaling_factor=TARGET_SCALING_FACTOR_GLOBAL)
plot_results_multi_horizon(history, y_valid_seq, valid_preds, dates_valid, 'Validation',
                           pred_horizon_to_plot=[1, 5, 20],
                           target_scaling_factor=TARGET_SCALING_FACTOR_GLOBAL)
plot_results_multi_horizon(history, y_test_seq, test_preds, dates_test, 'Test',
                           pred_horizon_to_plot=[1, 5, 20],
                           target_scaling_factor=TARGET_SCALING_FACTOR_GLOBAL)

# 方向精度を全てのホライズンでプロット
plot_direction_accuracy_multi_horizon(y_train_seq, train_preds, 'Train',
                                     target_scaling_factor=TARGET_SCALING_FACTOR_GLOBAL)
plot_direction_accuracy_multi_horizon(y_valid_seq, valid_preds, 'Validation',
                                     target_scaling_factor=TARGET_SCALING_FACTOR_GLOBAL)
plot_direction_accuracy_multi_horizon(y_test_seq, test_preds, 'Test',
                                     target_scaling_factor=TARGET_SCALING_FACTOR_GLOBAL)